In [ ]:
import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_dense_adj

# Generating a PPI-style graph (1,603 nodes, 2,757 edges) as per paper specs
num_nodes = 1603
num_edges = 2757
g = nx.gnm_random_graph(num_nodes, num_edges, seed=42)
adj = nx.adjacency_matrix(g).todense()
A = torch.tensor(adj).float()

In [ ]:
def get_normalized_adj(A):
    # Add self-loops (A_tilde = A + I)
    I = torch.eye(A.size(0))
    A_tilde = A + I
    
    # Calculate Degree Matrix (D_tilde)
    degrees = torch.sum(A_tilde, dim=1)
    D_tilde_inv_sqrt = torch.diag(torch.pow(degrees, -0.5))
    
    # Symmetric Normalization: D^-1/2 * A_tilde * D^-1/2
    return D_tilde_inv_sqrt @ A_tilde @ D_tilde_inv_sqrt

A_norm = get_normalized_adj(A)

In [ ]:
# Calculate Eigenvalues
eig_raw = torch.linalg.eigvals(A).abs()
eig_norm = torch.linalg.eigvals(A_norm).abs()

plt.figure(figsize=(10, 5))
plt.hist(eig_raw.numpy(), bins=50, alpha=0.5, label=f'Raw Adjacency (Max: {eig_raw.max():.2f})')
plt.hist(eig_norm.numpy(), bins=50, alpha=0.5, label=f'Normalized (Max: {eig_norm.max():.2f})')
plt.title("Spectral Radius Comparison: Numerical Stability Analysis")
plt.xlabel("Eigenvalue Magnitude")
plt.ylabel("Frequency")
plt.legend()
plt.show()